In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import pearsonr
from scipy import stats

In [2]:
def month_saved(data):
    m,n = data.shape
    month_sa = []
    for i in range(12):
        T = []
        for j in range(m):
            if j%12 == i and data['TA_F_MDS'][j] != -9999:
                T.append(data['TA_F_MDS'][j])
        if np.mean(T) >= 0:
            month_sa.append(i+1)
    return month_sa

In [29]:
type_data = pd.read_excel('C:/Users/111/Desktop/Fluxnet_sites.xlsx')
site_IGBP = type_data['IGBP'].values
site_ID =  type_data['SITE_ID'].values

path1 = 'D:/Data/Fluxnet/AllHourlyData_year3/'
path2 = 'D:/Data/Fluxnet/monthly_data/'

days1 = 15
days2 = 15

cor_VPD_RECO = ['VPD_RECO']
                             
for csv_file1 in os.listdir(path1):
    if csv_file1[4:10] in site_ID and site_IGBP[site_ID==csv_file1[4:10]] in  ['GRA', 'WSA', 'SAV', 'CRO', 'OSH', 'CSH']:
        if csv_file1[31:33] =='HH':
            hours = 48
        else:
            hours = 24
        for csv_file2 in os.listdir(path2):
            if csv_file1[4:10] == csv_file2[4:10]:
                print(csv_file1[4:10])
                data1 = pd.read_csv(path1+csv_file1)
                data2 = pd.read_csv(path2+csv_file2)
                month_sa = month_saved(data2)
                
                if len(month_sa)>=2:
                    m,n = data1.shape
                    month = np.round(data1['TIMESTAMP_START']/1000000).astype(int)%100
                    for i in range(int(m/hours/days2)-1):
                        #if month[i*days*hours]>=month_sa[0] and (month[(i+1)*days*hours]<=month_sa[-1]):
                        a = month[i*days2*hours:i*days2*hours+days1*hours]
                        b = np.array(month_sa)
                        if np.in1d(a, b).all():
                            Ti_0 = data1['TA_F_MDS'].values[i*days2*hours:i*days2*hours+days1*hours]
                            VPDi_0 = data1['VPD_F_MDS'].values[i*days2*hours:i*days2*hours+days1*hours]
                            T_QCi_0 = data1['VPD_F_MDS_QC'].values[i*days2*hours:i*days2*hours+days1*hours]
                            VPD_QCi_0 = data1['VPD_F_MDS_QC'].values[i*days2*hours:i*days2*hours+days1*hours]
                            RECOi_0 = data1['RECO_NT_VUT_REF'].values[i*days2*hours:i*days2*hours+days1*hours]

                            Ti_1 = Ti_0[(Ti_0!=-9999) & (VPDi_0!=-9999)  & (RECOi_0!=-9999) & (RECOi_0>0.01) & (T_QCi_0==0) & (VPD_QCi_0==0)& (Ti_0>5)]
                            VPDi_1 = VPDi_0[(Ti_0!=-9999) & (VPDi_0!=-9999) & (RECOi_0!=-9999) & (RECOi_0>0.01) & (T_QCi_0==0) & (VPD_QCi_0==0)& (Ti_0>5)]
                            RECOi_1 = RECOi_0[(Ti_0!=-9999) & (VPDi_0!=-9999) & (RECOi_0!=-9999) & (RECOi_0>0.01) & (T_QCi_0==0) & (VPD_QCi_0==0)& (Ti_0>5)]
                                    
                            if len(Ti_1) > 300:
                                SVPi_1 = 6.109*np.exp((17.625*Ti_1)/(243.04+Ti_1))
                                AVPi_1 = SVPi_1-VPDi_1
                                for Ti in np.arange(22,30,0.5):
                                    VPD_ik = AVPi_1[(Ti_1>=Ti-0.5) & (Ti_1<Ti+0.5) & (AVPi_1>27.4)]
                                    RECO_ik = RECOi_1[(Ti_1>=Ti-0.5) & (Ti_1<Ti+0.5) & (AVPi_1>27.4)]
                                    if len(VPD_ik)>30:
                                        VPD_ik_85th = np.percentile(VPD_ik,80)
                                        VPD_ik_15th = np.percentile(VPD_ik,20)
                                        if VPD_ik_85th-VPD_ik_15th>4:
                                            cor_VPD_RECO.append(stats.spearmanr(VPD_ik, RECO_ik)[0])
                                            
result = np.array([cor_VPD_RECO])
result_pd = pd.DataFrame(result.T)
save_path = './New_result/threshold2/Cor_non_forest_AVP_RECO_u_days30_cal3.csv'
result_pd.to_csv(save_path, index= 'a', header= False)

AT-Neu
AU-ASM
AU-Cpr
AU-DaP
AU-DaS
AU-Dry
AU-Gin
AU-How
AU-Rig
AU-Stp
BE-Lon
CA-NS6
CA-NS7
CA-SF3
CG-Tch
CH-Cha
CH-Fru
CH-Oe1
CH-Oe2
CN-Cng
CN-Du2
CN-Du3


D:\software\anaconda\envs\Plot\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
D:\software\anaconda\envs\Plot\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


CZ-BK2
DE-Geb
DE-Gri
DE-Kli
DE-RuR
DE-RuS
DE-Seh
DK-Eng
ES-Amo
ES-LJu
FI-Jok
FR-Gri
GL-ZaH
IT-BCi
IT-CA2
IT-MBo
IT-Noe
IT-Tor
NL-Hor
RU-Cok
SD-Dem
SN-Dhr
US-AR1
US-AR2
US-ARM
US-Cop
US-Goo
US-IB2
US-KS2
US-LWW
US-Ne1
US-Ne2
US-Ne3
US-SRC
US-SRG
US-SRM
US-Sta
US-Ton
US-Twt
US-Var
US-Whs
US-Wkg


In [26]:
type_data = pd.read_excel('C:/Users/111/Desktop/Fluxnet_sites.xlsx')
site_IGBP = type_data['IGBP'].values
site_ID =  type_data['SITE_ID'].values

path1 = 'D:/Data/Fluxnet/AllHourlyData_year3/'
path2 = 'D:/Data/Fluxnet/monthly_data/'

days1 = 25
days2 = 15

cor_VPD_RECO = ['VPD_RECO']

for csv_file1 in os.listdir(path1):
    if csv_file1[4:10] in site_ID and site_IGBP[site_ID==csv_file1[4:10]] in  ['DBF', 'EBF', 'DNF', 'ENF', 'MF']:
        if csv_file1[31:33] =='HH':
            hours = 48
        else:
            hours = 24
        for csv_file2 in os.listdir(path2):
            if csv_file1[4:10] == csv_file2[4:10]:
                print(csv_file1[4:10])
                
                data1 = pd.read_csv(path1+csv_file1)
                data2 = pd.read_csv(path2+csv_file2)
                month_sa = month_saved(data2)
                
                if len(month_sa)>=2:
                    m,n = data1.shape
                    month = np.round(data1['TIMESTAMP_START']/1000000).astype(int)%100
                    for i in range(int(m/hours/days)-1):
                        #if month[i*days*hours]>=month_sa[0] and (month[(i+1)*days*hours]<=month_sa[-1]):
                        a = month[i*days2*hours:i*days2*hours+days1*hours]
                        b = np.array(month_sa)
                        if np.in1d(a, b).all():
                            Ti_0 = data1['TA_F_MDS'].values[i*days2*hours:i*days2*hours+days1*hours]
                            VPDi_0 = data1['VPD_F_MDS'].values[i*days2*hours:i*days2*hours+days1*hours]
                            T_QCi_0 = data1['VPD_F_MDS_QC'].values[i*days2*hours:i*days2*hours+days1*hours]
                            VPD_QCi_0 = data1['VPD_F_MDS_QC'].values[i*days2*hours:i*days2*hours+days1*hours]
                            RECOi_0 = data1['RECO_NT_VUT_REF'].values[i*days2*hours:i*days2*hours+days1*hours]

                            Ti_1 = Ti_0[(Ti_0!=-9999) & (VPDi_0!=-9999)  & (RECOi_0!=-9999) & (RECOi_0>0.01) & (T_QCi_0==0) & (VPD_QCi_0==0)& (Ti_0>5)]
                            VPDi_1 = VPDi_0[(Ti_0!=-9999) & (VPDi_0!=-9999) & (RECOi_0!=-9999) & (RECOi_0>0.01) & (T_QCi_0==0) & (VPD_QCi_0==0)& (Ti_0>5)]
                            RECOi_1 = RECOi_0[(Ti_0!=-9999) & (VPDi_0!=-9999) & (RECOi_0!=-9999) & (RECOi_0>0.01) & (T_QCi_0==0) & (VPD_QCi_0==0)& (Ti_0>5)]
                                    
                            if len(Ti_1) > 300:    
                                SVPi_1 = 6.109*np.exp((17.625*Ti_1)/(243.04+Ti_1))
                                AVPi_1 = SVPi_1-VPDi_1
                                for Ti in np.arange(22,34,0.5):
                                    VPD_ik = AVPi_1[(Ti_1>=Ti-0.5) & (Ti_1<Ti+0.5) & (AVPi_1>25.84)]
                                    RECO_ik = RECOi_1[(Ti_1>=Ti-0.5) & (Ti_1<Ti+0.5) & (AVPi_1>25.84)]
                                    if len(VPD_ik)>30:
                                        VPD_ik_85th = np.percentile(VPD_ik,80)
                                        VPD_ik_15th = np.percentile(VPD_ik,20)
                                        if VPD_ik_85th-VPD_ik_15th>4:
                                            cor_VPD_RECO.append(stats.spearmanr(VPD_ik, RECO_ik)[0])

result = np.array([cor_VPD_RECO])
result_pd = pd.DataFrame(result.T)
save_path = './New_result/threshold2/Cor_forest_AVP_RECO_u_days30_cal3.csv'
result_pd.to_csv(save_path, index= 'a', header= False)

AR-Vir
AU-Tum
AU-Wac
AU-Whr
AU-Wom
BE-Bra
BE-Vie
BR-Sa1
BR-Sa3
CA-Gro
CA-Man
CA-NS1
CA-NS2
CA-NS3
CA-NS4
CA-NS5
CA-Oas
CA-Obs
CA-Qfo
CA-SF1
CA-SF2
CA-TP1
CA-TP2
CA-TP3
CA-TP4
CH-Dav
CH-Lae
CZ-BK1
DE-Hai
DE-Lkb
DE-Lnf
DE-Obe
DE-Tha
DK-Sor
FI-Hyy
FI-Let
FI-Sod
FR-Fon
FR-LBr
FR-Pue
GF-Guy
GH-Ank
IT-CA1
IT-CA3
IT-Col
IT-Cp2
IT-Cpz
IT-Lav
IT-Ren
IT-Ro1
IT-Ro2
IT-SRo
JP-SMF
MY-PSO
NL-Loo
RU-Fyo
US-Blo
US-GBT
US-GLE
US-Ha1
US-Me2
US-Me3
US-Me4
US-Me5
US-Me6
US-MMS
US-NR1
US-Oho
US-PFa
US-Prr
US-Syv
US-UMB
US-UMd
US-WCr
ZM-Mon
